# Parte 1: N-Rainhas
---

## Parâmetros editáveis

In [96]:
# Arquivos de jogos que serão processados
# game_files = ["games/game10.txt", "games/game20.txt", "games/game25.txt"]
game_files = ["games/game10.txt"]

# Tamanho dos quadrados do tabuleiro (visual)
# w, h = 30, 30

# Espaçamento entre os tabuleiros (visual)
# hspace, vspace = 40, 40

Antes de executar este caderno, pode ser necessário instalar previamente as dependências em seu sistema. Isso pode ser feito com os seguintes comandos:
```bash
pip install --user numpy
```

## Dependencias


In [97]:
import numpy as np
import random
from copy import copy, deepcopy

## Importando os dados

In [98]:
games = []

for game_file in game_files:
    game = np.loadtxt(game_file, dtype='str', delimiter=' ')
    games.append(game)

# Mostrar um game só de exemplo
games[0]

array([[&#39;-&#39;, &#39;-&#39;, &#39;-&#39;, &#39;-&#39;, &#39;-&#39;, &#39;-&#39;, &#39;-&#39;, &#39;-&#39;, &#39;-&#39;, &#39;-&#39;],
       [&#39;-&#39;, &#39;-&#39;, &#39;-&#39;, &#39;-&#39;, &#39;-&#39;, &#39;-&#39;, &#39;-&#39;, &#39;-&#39;, &#39;-&#39;, &#39;-&#39;],
       [&#39;-&#39;, &#39;-&#39;, &#39;-&#39;, &#39;-&#39;, &#39;-&#39;, &#39;-&#39;, &#39;-&#39;, &#39;-&#39;, &#39;-&#39;, &#39;-&#39;],
       [&#39;-&#39;, &#39;-&#39;, &#39;-&#39;, &#39;-&#39;, &#39;-&#39;, &#39;-&#39;, &#39;-&#39;, &#39;-&#39;, &#39;-&#39;, &#39;-&#39;],
       [&#39;-&#39;, &#39;-&#39;, &#39;-&#39;, &#39;-&#39;, &#39;-&#39;, &#39;-&#39;, &#39;-&#39;, &#39;-&#39;, &#39;-&#39;, &#39;-&#39;],
       [&#39;-&#39;, &#39;-&#39;, &#39;-&#39;, &#39;-&#39;, &#39;-&#39;, &#39;-&#39;, &#39;-&#39;, &#39;-&#39;, &#39;-&#39;, &#39;-&#39;],
       [&#39;-&#39;, &#39;-&#39;, &#39;-&#39;, &#39;-&#39;, &#39;-&#39;, &#39;-&#39;, &#39;-&#39;, &#39;-&#39;, &#39;-&#39;, &#39;-&#39;],
       [&#39;-&#39;, &#39;-

## Matriz auxiliar

In [99]:
matrix = {}

for i in range(0, len(game)):
    for j in range(0,len(game)):
        l = list()
        ## Add all line cells
        for k in range(0,len(game)):
            if k != j:
                l.append((i, k))
        ## Add all diagonal
        for k in range(0, len(game)):
            if k != j:
                dist = abs(k - j)
                if i + dist < len(game):
                    l.append((i + dist, k))
                if i - dist >= 0:
                    l.append((i - dist, k))
        matrix[(i, j)] = l

for cell in matrix:
    print (cell, "->", matrix[cell])


(0, 0) -&gt; [(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9), (1, 1), (2, 2), (3, 3), (4, 4), (5, 5), (6, 6), (7, 7), (8, 8), (9, 9)]
(0, 1) -&gt; [(0, 0), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9), (1, 0), (1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (6, 7), (7, 8), (8, 9)]
(0, 2) -&gt; [(0, 0), (0, 1), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9), (2, 0), (1, 1), (1, 3), (2, 4), (3, 5), (4, 6), (5, 7), (6, 8), (7, 9)]
(0, 3) -&gt; [(0, 0), (0, 1), (0, 2), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9), (3, 0), (2, 1), (1, 2), (1, 4), (2, 5), (3, 6), (4, 7), (5, 8), (6, 9)]
(0, 4) -&gt; [(0, 0), (0, 1), (0, 2), (0, 3), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9), (4, 0), (3, 1), (2, 2), (1, 3), (1, 5), (2, 6), (3, 7), (4, 8), (5, 9)]
(0, 5) -&gt; [(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 6), (0, 7), (0, 8), (0, 9), (5, 0), (4, 1), (3, 2), (2, 3), (1, 4), (1, 6), (2, 7), (3, 8), (4, 9)]
(0, 6) -&gt; [(0, 0), (0, 1), (0, 2), (0, 3), (0, 4)

In [100]:
class State():
    def __init__(self):
        self.queens = set()
        self.score = 0

## Returns a set of positions. Each position represents a queen
def generate_initial_state(game):
    game_size = len(game)
    state = State()
    for j in range(0, game_size):
        i = random.randrange(0, game_size)
        state.queens.add((i, j))
    
    # print(state.queens)
    evaluate_score(state)
    return state

## Evaluate score for a set of queens
def evaluate_score(state):
    state.score = 0
    for queen in state.queens:
        for neighbor_cell in matrix[queen]:
            if neighbor_cell in  state.queens:
                state.score += 1

## Randomly selects a queen from state, moves it and update state's score
def next_state(current_state):
    state = deepcopy(current_state)

    ## Select a queen
    queen = random.sample(state.queens, k=1)[0]

    ## Randomly selects new_queen
    while True:
        new_queen = (random.randrange(0, len(game)), queen[1])
        if new_queen != queen:
            break
    
    ## Remove queen
    state.queens.remove(queen)

    ## Update score
    for neighbor_cell in matrix[queen]:
        if neighbor_cell in state.queens:
            state.score -= 2
    
    ## Add new queen
    state.queens.add(new_queen)

    ## Update score
    for neighbor_cell in matrix[new_queen]:
        if neighbor_cell in state.queens:
            state.score += 2

    return state

def build_answer(state):
    board = copy(game)
    for queen in state.queens:
        board[queen] = 'Q'
    
    print(board, state.score)


## Hill Climbing


In [101]:
while True:
    current = generate_initial_state(game)
    plato = 20

    while True:
        nextState = next_state(current)
        # build_answer(nextState)

        if nextState.score == 0:
            break

        if nextState.score < current.score:
            current = nextState
            plato = 20
        else:
            plato -= 1
            if plato == 0:
                break

    if nextState.score == 0:
        break

build_answer(nextState)

[[&#39;-&#39; &#39;-&#39; &#39;-&#39; &#39;-&#39; &#39;-&#39; &#39;-&#39; &#39;-&#39; &#39;Q&#39; &#39;-&#39; &#39;-&#39;]
 [&#39;-&#39; &#39;-&#39; &#39;-&#39; &#39;-&#39; &#39;Q&#39; &#39;-&#39; &#39;-&#39; &#39;-&#39; &#39;-&#39; &#39;-&#39;]
 [&#39;-&#39; &#39;-&#39; &#39;Q&#39; &#39;-&#39; &#39;-&#39; &#39;-&#39; &#39;-&#39; &#39;-&#39; &#39;-&#39; &#39;-&#39;]
 [&#39;Q&#39; &#39;-&#39; &#39;-&#39; &#39;-&#39; &#39;-&#39; &#39;-&#39; &#39;-&#39; &#39;-&#39; &#39;-&#39; &#39;-&#39;]
 [&#39;-&#39; &#39;-&#39; &#39;-&#39; &#39;-&#39; &#39;-&#39; &#39;-&#39; &#39;Q&#39; &#39;-&#39; &#39;-&#39; &#39;-&#39;]
 [&#39;-&#39; &#39;-&#39; &#39;-&#39; &#39;-&#39; &#39;-&#39; &#39;-&#39; &#39;-&#39; &#39;-&#39; &#39;-&#39; &#39;Q&#39;]
 [&#39;-&#39; &#39;-&#39; &#39;-&#39; &#39;-&#39; &#39;-&#39; &#39;Q&#39; &#39;-&#39; &#39;-&#39; &#39;-&#39; &#39;-&#39;]
 [&#39;-&#39; &#39;-&#39; &#39;-&#39; &#39;-&#39; &#39;-&#39; &#39;-&#39; &#39;-&#39; &#39;-&#39; &#39;Q&#39; &#39;-&#39;]
 [&#39;-&#39; &#